In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
train_df = pd.read_csv('/kaggle/input/tabular-playground-series-mar-2021/train.csv')                      
test_df = pd.read_csv('/kaggle/input/tabular-playground-series-mar-2021/test.csv')                        
sample_submission = pd.read_csv('/kaggle/input/tabular-playground-series-mar-2021/sample_submission.csv') 

In [ ]:
train_df.head()

In [ ]:
test_df.head()

In [ ]:
sample_submission.head()

In [ ]:
train_df.info()

In [ ]:
all_df = pd.concat([train_df[train_df.columns[:-1]], test_df])
y = train_df['target']

In [ ]:
categorical_clm = all_df.columns[1:20]
categorical_clm

In [ ]:
for column in categorical_clm:
    print(all_df[column].value_counts().count())

In [ ]:
def onehot_encode(df, column):
    df = df.copy()
    dummies = pd.get_dummies(df[column], prefix=column, drop_first = True)
    df = pd.concat([df, dummies], axis=1)
    df = df.drop(column, axis=1)
    return df

In [ ]:
for column in categorical_clm:
    all_df = onehot_encode(all_df, column)

In [ ]:
all_df.head()

In [ ]:
train_df = all_df[:len(train_df)]
test_df = all_df[len(train_df):]

train_df.shape, test_df.shape

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(train_df,y, test_size = 0.2, random_state=51)

In [ ]:
X_train.head()

In [ ]:
y_train.head()

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.metrics import mean_absolute_error, accuracy_score
from xgboost import XGBClassifier

In [ ]:
models = [LogisticRegression(),
         DecisionTreeClassifier(),
         RandomForestClassifier(),
         GradientBoostingClassifier(),
         XGBClassifier()]

In [ ]:
best_model = ''
best_score = -100

for model in models:
    
    model.fit(X_train, y_train)
    predictions = model.predict(X_test)
    scores = accuracy_score(y_test, predictions)
    print(f"The model: {model}has following score: {scores}")
    
    if scores > best_score:
        best_score = scores
        best_model = model
        
print(f"\nThe best model is {best_model} it has the {best_score} scores")

In [ ]:
best_model = RandomForestClassifier(random_state = 51, n_estimators = 300)
best_model.fit(X_train, y_train)
predictions = best_model.predict(X_test)
scores = accuracy_score(y_test,predictions)
scores

In [ ]:
test_predictions = best_model.predict(test_df)
test_predictions 

In [ ]:
prediction_df = pd.DataFrame(data= test_predictions)
prediction_df.head()

In [ ]:
cols = prediction_df.columns.tolist()
cols = cols[-1:]+cols[:-1]
prediction_df = prediction_df[cols]
prediction_df.head()

In [ ]:
prediction_df['id']= test_df['id'].copy()
prediction_df.head(10)